# Regression Week 2: Multiple Regression (Interpretation)

In [1]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [3]:
from google.colab import files

In [4]:
uploaded = files.upload()

Saving kc_house_data.csv to kc_house_data.csv


In [ ]:
pip install turicreate

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [6]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [7]:
sales = turicreate.SFrame('kc_house_data.csv')

Finished parsing file /content/kc_house_data.csv

Parsing completed. Parsed 100 lines in 0.146972 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /content/kc_house_data.csv

Parsing completed. Parsed 21613 lines in 0.097481 secs.

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [8]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [9]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = turicreate.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                    validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training Max Error | Training Root-Mean-Square Error |

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 1.013183     | 4146407.590959     | 258679.804259                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [10]:
example_weight_summary = example_model.coefficients
print example_weight_summary

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0692068  | 7873.33813676 |
| sqft_living |  None | 315.403441118  | 3.45570032294 |
|   bedrooms  |  None | -65080.2154416 | 2717.45685213 |
|  bathrooms  |  None | 6944.02087631  | 3923.11492813 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [11]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

271789.504278


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [12]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    yhats = model.predict(data)
    # Then compute the residuals/errors
    residuals = yhats - outcome
    # Then square and add them up
    RSS = (residuals*residuals).sum()
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [13]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.73761534465e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [14]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [15]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [16]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [42]:
s1=test_data['bed_bath_rooms'].mean()
print(s1)

7.50390163159


In [43]:
s2=test_data['log_sqft_living'].mean()
print(s2)

7.55027467965


In [44]:
s3=test_data['bedrooms_squared'].mean()
print(s3)

12.4466777016


In [45]:
s4=test_data['lat_plus_long'].mean()
print(s4)

-74.653333554


In [28]:
pip install graphlab-create

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [29]:
feat1 = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
feat2 = feat1 + ['bed_bath_rooms']
feat3 = feat2 + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [33]:
# Learn the three models: (don't forget to set validation_set = None)
model1 = turicreate.linear_regression.create(train_data, target = 'price', features = feat1, validation_set = None, verbose=False)
model2 = turicreate.linear_regression.create(train_data, target = 'price', features = feat2, validation_set = None, verbose=False)
model3 = turicreate.linear_regression.create(train_data, target = 'price', features = feat3, validation_set = None, verbose=False)

In [38]:
# Examine/extract each model's coefficients:
coefficients = model1.coefficients
print(coefficients)

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | -56141852.4644 | 1649928.35804 |
| sqft_living |  None | 310.263506723  | 3.18882062328 |
|   bedrooms  |  None | -59577.6052121 | 2487.27455846 |
|  bathrooms  |  None |  13812.406876  | 3593.53440572 |
|     lat     |  None | 629863.119934  | 13120.6886699 |
|     long    |  None |  -214800.9523  | 13283.8203895 |
+-------------+-------+----------------+---------------+
[6 rows x 4 columns]



In [39]:
coefficients = model2.coefficients
print(coefficients)

+----------------+-------+----------------+---------------+
|      name      | index |     value      |     stderr    |
+----------------+-------+----------------+---------------+
|  (intercept)   |  None | -54412036.3966 | 1650346.09756 |
|  sqft_living   |  None | 304.449505239  | 3.20216518282 |
|    bedrooms    |  None | -116366.742818 |  4805.5339855 |
|   bathrooms    |  None | -77972.1056409 | 7565.03416078 |
|      lat       |  None | 625431.042062  | 13058.3308362 |
|      long      |  None | -203970.824936 | 13267.6490421 |
| bed_bath_rooms |  None | 26961.7302978  | 1956.35899367 |
+----------------+-------+----------------+---------------+
[7 rows x 4 columns]



In [40]:
coefficients = model3.coefficients
print(coefficients)

+------------------+-------+----------------+---------------+
|       name       | index |     value      |     stderr    |
+------------------+-------+----------------+---------------+
|   (intercept)    |  None | -52975504.214  | 1615141.28927 |
|   sqft_living    |  None |  529.19327225  | 7.69912257299 |
|     bedrooms     |  None |  28944.217163  | 9395.70086441 |
|    bathrooms     |  None | 65659.9030562  | 10795.3200464 |
|       lat        |  None |  704763.16512  | 745455237.283 |
|       long       |  None | -137781.471393 | 745455236.891 |
|  bed_bath_rooms  |  None | -8477.92763903 | 2858.94965517 |
| bedrooms_squared |  None | -6072.03763855 | 1494.96716541 |
| log_sqft_living  |  None | -563459.758053 | 17567.8176001 |
|  lat_plus_long   |  None | -83220.6812613 | 745455237.189 |
+------------------+-------+----------------+---------------+
[10 rows x 4 columns]



**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [31]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
# Compute the RSS on TRAINING data for each of the three models and record the values:
rss1 = get_residual_sum_of_squares(model1, train_data, train_data['price'])
rss2 = get_residual_sum_of_squares(model2, train_data, train_data['price'])
rss3 = get_residual_sum_of_squares(model3, train_data, train_data['price'])
print rss1, rss2, rss3

9.71323871612e+14 9.6158755258e+14 9.0527377473e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [32]:
# Compute the RSS on TESTING data for each of the three models and record the values:
# Compute the RSS on TESTING data for each of the three models and record the values:
rss1 = get_residual_sum_of_squares(model1, test_data, test_data['price'])
rss2 = get_residual_sum_of_squares(model2, test_data, test_data['price'])
rss3 = get_residual_sum_of_squares(model3, test_data, test_data['price'])
print rss1, rss2, rss3

2.26566318598e+14 2.24366990582e+14 2.51824565047e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.